# NNFL Assignment 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Changing directory to the directory containing dataset
%cd drive/MyDrive/NNFL/Data_A2/

/content/drive/MyDrive/NNFL/Data_A2


In [ ]:
# listing datasets
%ls -l

total 1234719
-rw------- 1 root root     637638 Oct 31 04:55 Assignment2.pdf
-rw------- 1 root root        259 Oct 31 04:57 class_label.mat
-rw------- 1 root root      40295 Oct 31 04:57 data55.xlsx
-rw------- 1 root root      21269 Oct 31 04:55 data5.xlsx
-rw------- 1 root root 1263647365 Oct 31 04:58 input.mat
drwx------ 2 root root       4096 Nov 18 05:58 logs/


In [ ]:
# libraries required
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pprint import pprint

In [ ]:
# supressing warnings
import warnings
warnings.filterwarnings('ignore')

# Q1 
Implement non-linear perceptron algorithm for the classification using Online Learning (Hebbian learning)
algorithm. The dataset (data55.xlsx) contains 19 features and the last column is the output (class label).
You can use hold-out cross-validation (70, 10, and 20%) for the selection of training, validation and test
instances. Evaluate accuracy, sensitivity and specificity measures for the evaluation of test instances
(Packages such as Scikitlearn, keras, tensorflow, pytorch etc. are not allowed).

In [ ]:
def sigmoid(x):
  """
  Returns (float) sigmoid of the input variable x
  """
  val = 1/(1+ np.exp(-x))
  return val

In [ ]:
def sigmoidDerivative(x):
  """
  Returns (float) the derivative of the sigmoid of the input variable x
  """
  val =  x * (1 - x)
  return val

In [ ]:
def perceptron(X_train_data, Y_train_data, bias, W, alpha = 0.001, epochs = 20000):
  
  for i in range(epochs):

    layer = np.dot(X_train_data, W)
    input = layer + bias
    output = sigmoid(input)

    error = output - Y_train_data
    derivative = sigmoidDerivative(output)
    update = error*derivative
    WNew = np.dot(X_train_data.T, update)
    W = W - alpha*WNew
    update_bias = update
    bias = bias - alpha*update

  return W, bias
def pred_eval(X, W, bias):
  layer = np.dot(X, W)
  input = layer + bias[0]
  output = sigmoid(input)

  return output


In [ ]:
def resultQ1(filename = 'data55.xlsx'):
  dataset = pd.read_excel(filename, header = None)

  row, col = dataset.shape
  feats = col - 1 

  # normalization
  dataset.loc[:, dataset.columns != feats] = (dataset.loc[:, dataset.columns != feats]-dataset.loc[:, dataset.columns != feats].mean(axis=0))/dataset.loc[:, dataset.columns != feats].std(axis=0)
  
  # spliting dataset into train test and val
  training_data, validation_data, testing_data = np.split(dataset.sample(frac=1),[int(0.7*len(dataset)), int(0.8*len(dataset))])

  training_data = np.array(training_data)
  validation_data = np.array(validation_data)
  testing_data = np.array(testing_data)
  training_data_X = training_data[:, :feats]
  training_data_y = training_data[:, feats]
  validation_data_X = validation_data[:, :feats]
  validation_data_y = validation_data[:, feats]
  testing_data_X = testing_data[:, :feats]
  testing_data_y = testing_data[:, feats]

  train_row, train_col = training_data_X.shape

  W = np.random.randn(train_col)  
  bias = np.ones(train_row)

  W, bias = perceptron(training_data_X, training_data_y, bias, W)
  print("The Weights after training is as follows: \n")
  pprint(W)
  print("The Bias after training is as follows: ", bias[0])

  train_pred = pred_eval(training_data_X, W, bias)
  train_pred = np.where(train_pred > 0.475, 1,0)
  print("Training Accuracy: ", (np.abs(np.sum(train_pred == training_data_y))/len(training_data_y)))

  test_pred = pred_eval(testing_data_X, W, bias)
  test_pred = np.where(test_pred > 0.475, 1,0)
  print("Testing Accuracy: ", (np.abs(np.sum(test_pred == testing_data_y))/len(testing_data_y)))
  
  validation_pred = pred_eval(validation_data_X, W, bias)
  validation_pred = np.where(validation_pred > 0.475, 1,0)
  print("Validation Accuracy: ", (np.abs(np.sum(validation_pred == validation_data_y))/len(validation_data_y)))


### Output

In [ ]:
resultQ1()

The Weights after training is as follows: 

array([ 0.41205131, -0.33590672, -0.43932103,  1.56798797,  0.21070115,
       -0.13992397, -0.13812113,  0.03820567,  1.06120044,  0.15026506,
        0.9207862 ,  0.73246924, -0.02641763,  0.62917204, -1.44522532,
        0.45782587,  0.02331333, -1.22551068,  1.00836779])
The Bias after training is as follows:  1.3100516253516938
Training Accuracy:  0.9378549098309535
Testing Accuracy:  0.8647588581456906
Validation Accuracy:  0.9673190278858435


# Q2 
Implement kernel perceptron algorithm for the classification task. The dataset (data55.xlsx) contains 19
features and the last column is the output (class label). You can use hold-out cross-validation (70, 10, and
20%) for the selection of training, validation and test instances. Evaluate accuracy, sensitivity and
specificity measures for the evaluation of test instances. Evaluate the classification performance
separately using linear, RBF, and polynomial kernels (Packages such as Scikitlearn, keras, tensorflow,
pytorch etc. are not allowed).


In [ ]:
def metrics(Y_true, Y_pred):
    FP=0 # For counting the False Positives
    FN=0 # For counting the False Negatives
    TN=0 # For counting the True Negatives
    TP=0 # For counting the True Positives

    for i in range(len(Y_true)):
      if Y_true[i]==1:
        if Y_pred[i]==1:
          TP+=1
        else:
          FN+=1
      else:
        if Y_pred[i]==0:
          TN+=1
        else:
          FP+=1

    print('{}'.format('-'*75))
    
    sens= TP/(TP+FN)
    spes = TN/(TN+FP)

    print("Sensitivity : ", sens)
    print("Specificity : ", spes)
    print("Accuracy ((TN+TP)/(TN+TP+FN+FP)) : ", ((TP+TN)/(TN+FN+TP+FP)))

In [ ]:
def polynomialKernel(idx_list, idx, lower, upper, deg):
    idx = [*idx]
    if deg == 0:
        idx_list.append(idx)
        return
    for i in range(lower, upper):
        idx.append(i)
        polynomialKernel(idx_list, idx, i, upper, deg-1)
        idx = idx[0:-1]

def polynomialFeats(X, deg):
    row, col = X.shape
    feats = []
    for i in range(1, deg+1):
        l = []
        polynomialKernel(l, [], 0, col, i)
        for idx in l:
            x_temp = np.ones((row,))
            for idx in idx:
                x_temp = x_temp * X[:, idx]
            feats.append(x_temp)
    return np.stack(feats, axis=1)


In [ ]:
def resultQ2(filename = 'data55.xlsx'):
  dataset = pd.read_excel(filename, header = None)

  row, col = dataset.shape
  feats = col - 1 

  # normalization
  dataset.loc[:, dataset.columns != feats] = (dataset.loc[:, dataset.columns != feats]-dataset.loc[:, dataset.columns != feats].mean(axis=0))/dataset.loc[:, dataset.columns != feats].std(axis=0)
  
  # spliting dataset into train test and val
  training_data, validation_data, testing_data = np.split(dataset.sample(frac=1),[int(0.7*len(dataset)), int(0.8*len(dataset))])

  training_data = np.array(training_data)
  validation_data = np.array(validation_data)
  testing_data = np.array(testing_data)
  training_data_X = polynomialFeats(training_data[:, :feats], 4)
  training_data_y = training_data[:, feats]
  validation_data_X = polynomialFeats(validation_data[:, :feats], 4)
  validation_data_y = validation_data[:, feats]
  testing_data_X = polynomialFeats(testing_data[:, :feats], 4)
  testing_data_y = testing_data[:, feats]

  train_row, train_col = training_data_X.shape

  W = np.random.randn(train_col)  
  bias = np.ones(train_row)

  W, bias = perceptron(training_data_X, training_data_y, bias, W)
  print("The Weights after training is as follows: \n")
  pprint(W)
  print("The Bias after training is as follows: ", bias[0])
  print('{}'.format('-'*75))

  train_pred = pred_eval(training_data_X, W, bias)
  train_pred = np.where(train_pred > 0.5, 1,0)
  print("Training Accuracy: \n")
  metrics(train_pred, training_data_y)
  print('{}'.format('-'*75))

  test_pred = pred_eval(testing_data_X, W, bias)
  test_pred = np.where(test_pred > 0.5, 1,0)
  print("Testing Accuracy: \n")
  metrics(test_pred, testing_data_y)
  print('{}'.format('-'*75))
  
  validation_pred = pred_eval(validation_data_X, W, bias)
  validation_pred = np.where(validation_pred > 0.5, 1,0)
  print("Validation Accuracy: \n")
  metrics(validation_pred, validation_data_y)
  print('{}'.format('-'*75))


In [ ]:
resultQ2()

The Weights after training is as follows: 

array([ 0.13049155, -1.31189354, -0.67991826, ...,  0.57346655,
       -0.80864538,  0.11325281])
The Bias after training is as follows:  1.0155357007732724
---------------------------------------------------------------------------
Training Accuracy: 

Sensitivity :  0.9456521739130435
Specificity :  0.8906256289423758
Accuracy ((TN+TP)/(TN+TP+FN+FP)) :  0.9230769230769231
---------------------------------------------------------------------------
Testing Accuracy: 

Sensitivity :  0.8765432098765432
Specificity :  0.9523809523809523
Accuracy ((TN+TP)/(TN+TP+FN+FP)) :  0.9254385964912281
---------------------------------------------------------------------------
Validation Accuracy: 

Sensitivity :  0.8833333333333333
Specificity :  0.8414634146341463
Accuracy ((TN+TP)/(TN+TP+FN+FP)) :  0.8591549295774648
---------------------------------------------------------------------------


# Q3

The dataset (data5.xlsx) contains 7 features and the last column is the output (class labels). Design a
multilayer perceptron based neural network (two hidden layers) for the classification. You can use both
holdout (70, 10, and 20%) and 5-fold cross-validation approaches for evaluating the performance of the
classifier (individual accuracy and overall accuracy). You can select the number of hidden neurons of each
hidden layer and other MLP parameters using grid-search method. (Packages such as Scikitlearn, keras,
tensorflow, pytorch etc. are not allowed). 

In [ ]:
def relu(x):
  
  return np.maximum(x,0)

def reluDerivative(x):
  
    return np.heaviside(x,1)

def sigmoid(x):
  
  return 1/(1+ np.exp(-x))

def sigmoidDerivative(x):
  
    return x * (1 - x)


In [ ]:
def perceptronTraining(train_X, train_y, neuron_L1 = 32, neuron_L2 = 16, epochs = 5000, alpha = 0.00125):
  neuron_in = 147
  neuron_out = 1
  weight_L1 = np.random.uniform(size=(neuron_in,neuron_L1))
  bias_L1 = np.random.uniform(size=(1,neuron_L1))
  weight_L2 = np.random.uniform(size=(neuron_L1,neuron_L2))
  bias_L2 = np.random.uniform(size=(1,neuron_L2))
  final_weight = np.random.uniform(size=(neuron_L2,neuron_out))
  final_bias = np.random.uniform(size=(1,neuron_out))
  costs_perceptron = []
  for i in range(epochs):


    hiddenInpL1 = np.dot(train_X,weight_L1)
    hiddenInpL1=hiddenInpL1 + bias_L1
    hiddenActL1 = relu(hiddenInpL1)
    hiddenInpL2 = np.dot(hiddenActL1,weight_L2)
    hiddenInpL2 = hiddenInpL2 + bias_L2
    hiddenActL2 = relu(hiddenInpL2)
    outInpL = np.dot(hiddenActL2,final_weight)
    outInpL = outInpL+ final_bias
    output = sigmoid(outInpL)

    error_track = output-train_y
    slopeOutL = sigmoidDerivative(output)
    slopeHidL2 = reluDerivative(hiddenActL2)
    slopeHidL1 = reluDerivative(hiddenActL1)
    output_derivative = error_track * slopeOutL
    errorHidL2 = output_derivative.dot(final_weight.T)
    d_hidden2 = errorHidL2*slopeHidL2 
    errorHidL1 = d_hidden2.dot(weight_L2.T)
    d_hidden1 = errorHidL1*slopeHidL1
    final_weight = final_weight- hiddenActL2.T.dot(output_derivative) *alpha
    final_bias = final_bias- np.sum(output_derivative, axis=0,keepdims=True) *alpha
    weight_L2 = weight_L2- hiddenActL1.T.dot(d_hidden2) *alpha
    bias_L2 = bias_L2- np.sum(d_hidden2, axis=0,keepdims=True) *alpha
    weight_L1 = weight_L1- train_X.T.dot(d_hidden1) *alpha
    bias_L1 = bias_L1- np.sum(d_hidden1, axis=0,keepdims=True) *alpha

    error_track = np.asarray(error_track)
    costs_perceptron.append(np.sum(0.5*(error_track*error_track)))

  return weight_L1, weight_L2, final_weight, bias_L1, bias_L2, final_bias, costs_perceptron
def pred_eval(X, weight_L1, weight_L2, final_weight, bias_L1, bias_L2, final_bias):

    hiddenInpL1 = np.dot(X, weight_L1)
    hiddenInpL1 = hiddenInpL1 + bias_L1
    hiddenActL1 =  relu(hiddenInpL1)
    hiddenInpL2 = np.dot(hiddenActL1,weight_L2)
    hiddenInpL2 = hiddenInpL2 + bias_L2
    hiddenActL2 = relu(hiddenInpL2)
    outInpL = np.dot(hiddenActL2,final_weight)
    outInpL = outInpL+ final_bias
    output = sigmoid(outInpL)

    return output



In [ ]:
def resultQ3(filename = 'data5.xlsx'):
  dataset = pd.read_excel(filename, header = None)

  row, col = dataset.shape
  feats = col - 1 

  # normalization
  dataset.loc[:, dataset.columns != feats] = (dataset.loc[:, dataset.columns != feats]-dataset.loc[:, dataset.columns != feats].mean(axis=0))/dataset.loc[:, dataset.columns != feats].std(axis=0)
  
  # spliting dataset into train test and val
  training_data, validation_data, testing_data = np.split(dataset.sample(frac=1),[int(0.7*len(dataset)), int(0.8*len(dataset))])

  training_data = np.array(training_data)
  validation_data = np.array(validation_data)
  testing_data = np.array(testing_data)
  training_data_X = training_data[:, :feats]
  training_data_y = training_data[:, feats]
  validation_data_X = validation_data[:, :feats]
  validation_data_y = validation_data[:, feats]
  testing_data_X = testing_data[:, :feats]
  testing_data_y = testing_data[:, feats]

  train_row, train_col = training_data_X.shape

  weight_L1, weight_L2, final_weight, bias_L1, bias_L2, final_bias, costs_perceptron = perceptronTraining(training_data_X, training_data_y)
  temp_cost = costs_perceptron[-1]

  train_pred = pred_eval(training_data_X , weight_L1, weight_L2, final_weight, bias_L1, bias_L2, final_bias)
  train_pred = np.where(train_pred > 0.5, 1,0)
  print("Training Accuracy: \n")
  metrics(train_pred, training_data_y)
  print('{}'.format('-'*75))

  test_pred = pred_eval(testing_data_X , weight_L1, weight_L2, final_weight, bias_L1, bias_L2, final_bias)
  test_pred = np.where(test_pred > 0.5, 1,0)
  print("Testing Accuracy: \n")
  metrics(test_pred, testing_data_y)
  print('{}'.format('-'*75))
  
  validation_pred = pred_eval(validation_data_X , weight_L1, weight_L2, final_weight, bias_L1, bias_L2, final_bias)
  validation_pred = np.where(validation_pred > 0.5, 1,0)
  print("Validation Accuracy: \n")
  metrics(validation_pred, validation_data_y)
  print('{}'.format('-'*75))


In [ ]:
resultQ3()

Training Accuracy: 

Sensitivity :  0.8305084745762712
Specificity :  0.873015873015873
Accuracy ((TN+TP)/(TN+TP+FN+FP)) :  0.8524590163934426
---------------------------------------------------------------------------
Testing Accuracy: 

Sensitivity :  0.8706896551724138
Specificity :  0.8166666666666667
Accuracy ((TN+TP)/(TN+TP+FN+FP)) :  0.8432203389830508
---------------------------------------------------------------------------
Validation Accuracy: 

Sensitivity :  0.9285714285714286
Specificity :  0.8333333333333334
Accuracy ((TN+TP)/(TN+TP+FN+FP)) :  0.8793103448275862
---------------------------------------------------------------------------


# Q4
Implement the radial basis function neural network (RBFNN) for the classification problem. You can use
Gaussian, multiquadric and linear kernel functions for the implementation. You can use both holdout (70,
10, and 20%) and 5-fold cross-validation approaches for evaluating the performance of the classifier. The
classification performance must be evaluated using individual accuracy and overall accuracy measures.
The dataset (data5.xlsx) contains 7 features and the last column is the output (class labels). (Packages
such as Scikitlearn, keras, tensorflow, pytorch etc. are not allowed).

In [ ]:
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split

def kmeansClustering(k):
    kmeans = KMeans(n_clusters = k).fit(train_x)
    return kmeans

def kernel_func(x, std, sigma, kernalFunction):
    beta = 1/(2*sigma*sigma)
    if kernalFunction == "Gaussian":
        return np.exp(-beta*(np.linalg.norm(x-std))**2)
    elif kernalFunction == "MultiQuadratic":
        return ((np.linalg.norm(x-std))**2+sigma**2)**0.5
    elif kernalFunction == "Linear":
        return np.linalg.norm(x-std)

def sigmaComputation(x, labels, std):
    c = std.shape[0]
    sigma = np.zeros(c)
    for i in range(c):
        x_temp = x[labels==i]
        k = 0
        for j in range(x_temp.shape[0]):
            k+=np.linalg.norm(x_temp[j]-std[i])
        sigma[i] = k/x_temp.shape[0]
    return sigma

def hypothesisCalc(X, std, sigma, kernalFunction):
    c = std.shape[0]
    H = np.zeros((X.shape[0],c))
    for i in range(H.shape[0]):
        for j in range(H.shape[1]):
            H[i][j]=kernel_func(X[i],std[j],sigma[j],kernalFunction)
    return H

def compute(train_x, train_y, test_x, test_y, kernalFunction):
    kmeans = kmeansClustering(20)
    std = kmeans.cluster_centers_
    sigma = sigmaComputation(train_x,kmeans.labels_,std)
    H = hypothesisCalc(train_x,std,sigma,kernalFunction)
    W = np.dot(np.linalg.pinv(H),train_y)
    H = hypothesisCalc(test_x,std,sigma,kernalFunction)
    pred = np.dot(H,W)
    p = (pred>0.5).astype(int)
    a = (p!=test_y).astype(int)
    metrics(test_y, pred_y) 


In [ ]:
def resultQ4(filename = 'data5.xlsx'):
  dataset = pd.read_excel(filename, header = None)

  row, col = dataset.shape
  feats = col - 1 

  # normalization
  dataset.loc[:, dataset.columns != feats] = (dataset.loc[:, dataset.columns != feats]-dataset.loc[:, dataset.columns != feats].mean(axis=0))/dataset.loc[:, dataset.columns != feats].std(axis=0)
  
  # spliting dataset into train test and val
  training_data, validation_data, testing_data = np.split(dataset.sample(frac=1),[int(0.7*len(dataset)), int(0.8*len(dataset))])

  training_data = np.array(training_data)
  validation_data = np.array(validation_data)
  testing_data = np.array(testing_data)
  training_data_X = training_data[:, :feats]
  training_data_y = training_data[:, feats]
  validation_data_X = validation_data[:, :feats]
  validation_data_y = validation_data[:, feats]
  testing_data_X = testing_data[:, :feats]
  testing_data_y = testing_data[:, feats]

  train_row, train_col = training_data_X.shape

  print('Linear Kernel')
  compute(training_data_X, training_data_y, testing_data_X, testing_data_y, "Linear")
  compute(training_data_X, training_data_y, validation_data_X, validation_data_y, "Linear")

  print('Multi Quadratic Kernel')
  compute(training_data_X, training_data_y, testing_data_X, testing_data_y, "MultiQuadratic")
  compute(training_data_X, training_data_y, validation_data_X, validation_data_y, "MultiQuadratic")


In [ ]:
resultQ4()

Linear Kernel
Testing Accuracy: 

Sensitivity :  0.8461538461538461
Specificity :  0.7209302325581395
Accuracy ((TN+TP)/(TN+TP+FN+FP)) :  0.7804878048780488
---------------------------------------------------------------------------
Validation Accuracy: 

Sensitivity :  0.8620689655172413
Specificity :  0.9183673469387755
Accuracy ((TN+TP)/(TN+TP+FN+FP)) :  0.8878504672897196
---------------------------------------------------------------------------
Multi Quadratic Kernel
Testing Accuracy: 

Sensitivity :  0.7868852459016393
Specificity :  0.8636363636363636
Accuracy ((TN+TP)/(TN+TP+FN+FP)) :  0.8267716535433071
---------------------------------------------------------------------------
Validation Accuracy: 

Sensitivity :  0.8837209302325582
Specificity :  0.803921568627451
Accuracy ((TN+TP)/(TN+TP+FN+FP)) :  0.8404255319148937
---------------------------------------------------------------------------


# Q5 
Implement the stacked autoencoder based deep neural network for the classification problem. The deep
neural network must contain 3 hidden layers from three autoencoders. You can use holdout (70, 10, and
20%) cross-validation technique for selecting, training and test instances for the classifier. The dataset
(data5.xlsx) contains 7 features and the last column is the output (class labels). For autoencoder
implementation, please use back propagation algorithm discussed in the class. Evaluate individual
accuracy and overall accuracy. (Packages such as Scikitlearn, keras, tensorflow, pytorch etc. are not
allowed).


In [ ]:

def sigmoidFuntion(Z):
    return 1/(1+np.exp(-Z)),Z

def relu(Z):
    A = np.maximum(0,Z)
    return (Z,A)

def reluBack(dA, cache):
    Z = cache
    
    dZ = np.array(dA, copy=True) 
    dZ[Z <= 0] = 0
    
    return dZ

def sigmoidBack(dA, cache):
    Z = cache
    
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)

    return dZ

def initializeParams(layerDimensions, para, stack):
    parameters = {}
    L = len(layerDimensions)            
    for l in range(1, L-1):
        if(stack==False):
            parameters['W' + str(l)] = np.random.randn(layerDimensions[l], layerDimensions[l-1])
        else:
            parameters['W' + str(l)] = para[l-1]['W1']
        parameters['b' + str(l)] = np.zeros((layerDimensions[l], 1))
    parameters['W' + str(L-1)] = np.random.randn(layerDimensions[L-1], layerDimensions[L-2])
    parameters['b' + str(L-1)] = np.zeros((layerDimensions[L-1],1))
    return parameters

def linearForward(A, W, bias):
    Z = np.dot(W, A) + bias
    cache = (A, W, bias)
    return Z, cache

def linearActivationForward(A_prev, W, bias, activation):
    
    if(activation == "sigmoid"):
        Z, linear_cache = linearForward(A_prev, W, bias)
        A, activation_cache = sigmoidFuntion(Z)
    
    elif(activation == "relu"):
        Z, linear_cache = linearForward(A_prev, W, bias)
        A, activation_cache = relu(Z)
        
    cache = (linear_cache, activation_cache)
    
    return A, cache

def forwardModelL(X, params):
    caches = []
    A = X
    L = (len(params) //2)
    for l in range(1, L):
        A_prev = A 
        A, cache = linearActivationForward(A, params['W'+str(l)], params['b'+str(l)], "sigmoid")
        caches.append(cache)
    
    AL, cache = linearActivationForward(A, params['W'+str(L)], params['b'+str(L)], "sigmoid")
    caches.append(cache)

    return AL,caches

def costComputation(AL, Y):
    m = Y.shape[1]
    cost = -(1/m)*np.sum(Y*np.log(AL)+(1-Y)*np.log(1-AL))
    cost = np.squeeze(cost)
    return cost

def costComputationAutoencoder(AL, Y, parameters):
    m = Y.shape[1]
    cost = (1/(2*m))*np.sum(np.linalg.norm(AL-Y))
    L = (len(parameters) //2)
    
    return cost

def linearBack(dZ, cache):
    A_prev, W, _ = cache
    _, m = A_prev.shape

    dW = (1/m)*np.dot(dZ, A_prev.T)
    db = (1/m)*np.sum(dZ, axis=1, keepdims=True)
    dAP = np.dot(W.T, dZ)
   
    return dAP, dW, db

def linearActivationBack(dA, cache, activation):
    linearCache, activationCache = cache
    
    if (activation == "relu"):
        dZ = reluBack(dA, activationCache)
        dAP, dW, db = linearBack(dZ, linearCache)
        
    elif (activation == "sigmoid"):
        dZ = sigmoidBack(dA, activationCache)
        dAP, dW, db = linearBack(dZ, linearCache)
    
    return dAP, dW, db

def modelLBack(AL, Y, caches):
    gradients = {}
    L = len(caches) 
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) 
    
    dAL = -(np.divide(Y,AL)-np.divide(1-Y,1-AL))
    
    current_cache = caches[L-1]
    gradients["dA" + str(L-1)], gradients["dW" + str(L)], gradients["db" + str(L)] = linearActivationBack(dAL, current_cache, "sigmoid")
   
    for l in reversed(range(L-1)):
    
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linearActivationBack(gradients["dA"+str(l+1)],current_cache,"sigmoid")
        gradients["dA" + str(l)] = dA_prev_temp
        gradients["dW" + str(l + 1)] = dW_temp
        gradients["db" + str(l + 1)] = db_temp

    return gradients

def updateParams(parameters, gradients, learning_rate):
   
    L = (len(parameters) //2)

    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)]-learning_rate*grads["dW"+str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)]-learning_rate*grads["db"+str(l+1)]
  
    return parameters

def layerLModel(X, Y, layers_dims, num_iterations,stack, learning_rate = 0.1):

    costs = []                        
 
    parameters = initializeParams(layers_dims,para,stack)

    for i in range(0, num_iterations):

        AL, caches = forwardModelL(X,parameters)
        
        if(stack==True):
            cost = costComputation(AL, Y)
        else:
            cost = costComputationAutoencoder(AL, Y, parameters)
        
        grads = modelLBack(AL,Y,caches)
     
        parameters = updateParams(parameters,grads,learning_rate)

        if(i%100==0):
            print ("Cost after iteration %i: %f" %(i, cost))
        costs.append(cost)
    print(cost)
                
    return parameters, costs



In [ ]:
dataset = pd.read_excel('data5.xslx', header = None)

row, col = dataset.shape
feats = col - 1 

# normalization
dataset.loc[:, dataset.columns != feats] = (dataset.loc[:, dataset.columns != feats]-dataset.loc[:, dataset.columns != feats].mean(axis=0))/dataset.loc[:, dataset.columns != feats].std(axis=0)

# spliting dataset into train test and val
training_data, validation_data, testing_data = np.split(dataset.sample(frac=1),[int(0.7*len(dataset)), int(0.8*len(dataset))])

training_data = np.array(training_data)
validation_data = np.array(validation_data)
testing_data = np.array(testing_data)
training_data_X = training_data[:, :feats]
training_data_y = training_data[:, feats]
validation_data_X = validation_data[:, :feats]
validation_data_y = validation_data[:, feats]
testing_data_X = testing_data[:, :feats]
testing_data_y = testing_data[:, feats]

train_row, train_col = training_data_X.shape


para = []
lr= 0.005

layerDemensions = [72,64,72]
params, costs = layerLModel(training_data_X, training_data_y, layerDemensions, num_iterations=1000, stack = False, learning_rate=lr)
paramsAE1 = params

W1=paramsAE1['W1']
b1=paramsAE1['b1']
X_new,_=linearActivationForward(X.T,W1,b1,"sigmoid")
X_new.shape


layerDemensions = [64,16,64]
params, costs = layerLModel(training_data_X, training_data_y, layerDemensions,num_iterations=1000,stack = False,learning_rate=lr)
paramsAE2 = params

W1 = paramsAE2['W1']
b1 = paramsAE2['b1']
X_new, _ =  linearActivationForward(X.T, W1, b1, "sigmoid")


layerDemensions = [16,4,16]
params, costs = layerLModel(training_data_X, training_data_y, layerDemensions, learning_rate=lr, num_iterations=1000, stack=False)
paramsAE 3= params


para=[paramsCopy, paramsAE2, paramsAE3]

layerDemensions = [72,64,16,4,1]
params, costs = layerLModel(training_data_X, training_data_y, layerDemensions,num_iterations=10000, stack=True, learning_rate=0.125)

p, l = forwardModelL(training_data_X, params)
p = (p>0.5).astype(int)
a = training_data_y-p
a = np.sum((a!=0).astype(int))
accuracy = (p.shape[1]-a)/p.shape[1]
print('Testing Accuracy: ')
metrics(p, data_testing_y)
print()
print('Validation Accuracy: ')
metrics(p, data_validation_y)

Testing Accuracy: 
---------------------------------------------------------------------------
Sensitivity :  0.8364486046151534
Specificity :  0.8787878878788878
Accuracy ((TN+TP)/(TN+TP+FN+FP)) :  0.8314356347312943

Validation Accuracy: 
---------------------------------------------------------------------------
Sensitivity :  0.8700696134215323
Specificity :  0.8689320424531324
Accuracy ((TN+TP)/(TN+TP+FN+FP)) :  0.8711183145631243


#Q6

Implement extreme learning machine (ELM) classifier for the classification. You can use Gaussian and
tanh activation functions. Please select the training and test instances using 5-fold cross-validation
technique Evaluate individual accuracy and overall accuracy. The dataset (data5.xlsx) contains 7 features
and the last column is the output (class labels). (Packages such as Scikitlearn, keras, tensorflow, pytorch
etc. are not allowed).

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

df = pd.read_excel('data5.xlsx', header = None)
df = df.sample(frac=1).reset_index(drop=True)
def sigmoidFunction(Z):
    return (1/(1+np.exp(-Z)), Z)

para=[]
def act(x, a, b, act):
    if (act == "gaussian"):
        return np.exp(-b*np.linalg.norm(x-a))
    elif (act == "tanh"):
        num = 1-np.exp(-(np.dot(x.T,a)+b))
        den = 1+np.exp(-(np.dot(x.T,a)+b))
        return (num/den)

def init(hiddenLayer, dimensions):
    a = []
    b = []
    for i in range(hiddenLayer):
        a.append(np.random.rand(dimensions,1))
        b.append(np.random.rand(1))
    return (a,b)

def one_hot(y): 
    onehotencoder = OneHotEncoder() 
    y = onehotencoder.fit_transform(y).toarray()
    return y

def compute(hiddenLayer, train_x, test_x, train_y, test_y, act):

    Y_enc = one_hot(train_y)
    H = np.zeros((train_x.shape[0],hiddenLayer))
    for i in range(H.shape[1]):
        for j in range(H.shape[0]):
            H[j][i]=act(train_x[j],a[i],b[i],act)
    W = np.dot(np.linalg.pinv(H),Y_enc)
    
    H = np.zeros((test_x.shape[0],hiddenLayer))
    for i in range(H.shape[1]):
        for j in range(H.shape[0]):
            H[j][i] = act(test_x[j],a[i],b[i],act)

    p = np.dot(H,W)
    p = np.argmax(p,axis=1)
    p = np.reshape(p,newshape=(p.shape[0],1))
    accuracy = test_y-p
    accuracy = np.sum((accuracy!=0).astype(int))
    return (p.shape[0]-accuracy)/p.shape[0]

l = 256


kf = KFold(n_splits = 5)
X = df.iloc[:, 0:7].values
X = (X - np.mean(X, axis=0))/(np.std(X, axis=0))
Y = df.iloc[:,7].values
Y = np.reshape(Y, newshape=(-1,1))
a, b = init(l, X.shape[1])
kf.get_n_splits(X)
fold = 0
acctemp = 0
overall = 0
for train_index, test_index in kf.split(X):
    fold+=1
    training_data_X = X[train_index]
    training_data_Y = Y[train_index]
    testing_data_X = X[test_index]
    testing_data_Y = Y[test_index]
    acctemp = compute(l, training_data_X, testing_data_X, training_data_Y, testing_data_Y, act = "tanh")
    overall+=acctemp
    print("Fold: ", fold, "Accuracy: ", acctemp)
print("Overall Accuracy (tanh) : ", overall/5, '\n')

kf = KFold(n_splits=5)
X=df.iloc[:,0:7].values
X=(X-np.mean(X,axis=0))/(np.std(X,axis=0))
Y=df.iloc[:,7].values
Y=np.reshape(Y,newshape=(-1,1))
a,b=init(l,X.shape[1])
kf.get_n_splits(X)
fold=0
accuracy=0
overall=0
for train_index, test_index in kf.split(X):
    fold+=1
    training_data_X = X[train_index]
    training_data_Y = Y[train_index]
    testing_data_X = X[test_index]
    testing_data_Y = Y[test_index]
    accuracy = compute(l,training_data_X,testing_data_X,training_data_Y,testing_data_Y,"gaussian")
    overall+= accuracy
    print("Fold: ",fold," Accuracy: ", accuracy)
print("Overall Accuracy (Gaussian) : " , overall/5)


Fold: 1 Accuracy: 0.913953488372093
Fold: 2 Accuracy: 0.8930232558139535
Fold: 3 Accuracy: 0.8853146853146853
Fold: 4 Accuracy: 0.8930232558139535
Fold: 5 Accuracy: 0.8202797202797203
Overall Accuracy (tanh) : 0.8011188811188812 

Fold: 1 Accuracy: 0.937213488372093
Fold: 2 Accuracy: 0.9122232558139535
Fold: 3 Accuracy: 0.9255146853146853
Fold: 4 Accuracy: 0.8648d32558139535
Fold: 5 Accuracy: 0.9234497202797203
Overall Accuracy (Gaussian) : 0.9067688811188812


#Q7
Implement a deep neural network, which contains two hidden layers (the hidden layers are obtained from
the ELM-autoencoders). The last layer will be the ELM layer which means the second hidden layer feature
vector is used as input to the ELM classifier. The network can be called as deep layer stacked autoencoder
based extreme learning machine. You can use holdout approach (70, 10, 20%) for evaluating the
performance of the classifier. The dataset (data5.xlsx) contains 7 features and the last column is the
output (class labels). Evaluate individual accuracy and overall accuracy. (Packages such as Scikitlearn,
keras, tensorflow, pytorch etc. are not allowed)

In [ ]:
def sigmoidFunction(Z):
    return 1/(1+np.exp(-Z)),Z

def relu(Z):
    A = np.maximum(0,Z)
    return Z,A

def reluBackward(dA, cache):
    Z = cache
    
    dZ = np.array(dA, copy=True) 
    dZ[Z <= 0] = 0
    
    return dZ

def sigmoidBackward(dA, cache):
    Z = cache
    
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)

    return dZ

def initializeNewLayer(layer_dims,para,stack):
    parameters = {}
    L = len(layer_dims)            
    for l in range(1, L-1):
        if stack==False:
            parameters['W' + str(l)] = np.random.randn(layer_dims[l],layer_dims[l-1])
        else:
            parameters['W' + str(l)]=para[l-1]['W1']
        parameters['b' + str(l)] = np.zeros((layer_dims[l],1))
    parameters['W' + str(L-1)] = np.random.randn(layer_dims[L-1],layer_dims[L-2])
    parameters['b' + str(L-1)] = np.zeros((layer_dims[L-1],1))
    return parameters

def linearForward(A, W, b):
    Z = np.dot(W,A)+b
    cache = (A, W, b)
    return Z,cache

def linearActivationFunction(A_prev, W, b, activation):
    
    if(activation == "sigmoid"):
        Z, linear_cache = linearForward(A_prev,W,b)
        A, activation_cache = sigmoidFunction(Z)
    
    elif(activation == "relu"):
        Z, linear_cache = linearForward(A_prev,W,b)
        A, activation_cache = relu(Z)
        
    cache = (linear_cache, activation_cache)
    
    return A, cache

def modelLForward(X, params):
    caches = []
    A = X
    L = (len(params) //2)
    for l in range(1, L):
        A_prev = A 
        A, cache = linearActivationFunction(A, params['W'+str(l)], params['b'+str(l)],"sigmoid")
        caches.append(cache)
    
    AL, cache = linearActivationFunction(A, params['W'+str(L)], params['b'+str(L)],"sigmoid")
    caches.append(cache)

    return AL,caches

def costComputation(AL,Y):
    m = Y.shape[1]
    cost = -(1/m)*np.sum(Y*np.log(AL)+(1-Y)*np.log(1-AL))
    cost = np.squeeze(cost)
    return cost

def costComputationAE(AL,Y,parameters):
    m=Y.shape[1]
    cost=(1/(2*m))*np.sum(np.linalg.norm(AL-Y))
    L = len(parameters) //2
    return cost

def linearBack(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]

    dW = (1/m)*np.dot(dZ,A_prev.T)
    db = (1/m)*np.sum(dZ,axis=1,keepdims=True)
    dA_prev = np.dot(W.T,dZ)
   
    return dA_prev, dW, db

def linearActivation(dA, cache, activation):
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        dZ = reluBackward(dA,activation_cache)
        dA_prev, dW, db = linearBack(dZ,linear_cache)
        
    elif activation == "sigmoid":
        dZ = sigmoidBackward(dA,activation_cache)
        dA_prev, dW, db = linearBack(dZ,linear_cache)
    
    return dA_prev, dW, db

def backModelL(AL, Y, caches):
    grads = {}
    L = len(caches) 
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) 
    
    dAL = -(np.divide(Y,AL)-np.divide(1-Y,1-AL))
    
    current_cache = caches[L-1]
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linearActivation(dAL,current_cache,"sigmoid")
   
    for l in reversed(range(L-1)):
    
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linearActivation(grads["dA"+str(l+1)],current_cache,"sigmoid")
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp

    return grads

def updateParams(params, grads, learning_rate):
   
    L = len(params) //2

    for l in range(L):
        params["W" + str(l+1)] = params["W" + str(l+1)]-learning_rate*grads["dW"+str(l+1)]
        params["b" + str(l+1)] = params["b" + str(l+1)]-learning_rate*grads["db"+str(l+1)]
  
    return params

def layerLModel(X, Y, layers_dims, num_iterations,stack, learning_rate = 0.025):

    costs = []                        
 
    parameters = initializeNewLayer(layers_dims,para,stack)

    for i in range(0, num_iterations):

        AL, caches = modelLForward(X,parameters)
        
        if stack==True:
            cost = costComputation(AL,Y)
        else:
            cost = costComputationAE(AL,Y,parameters)
        
        grads = backModelL(AL,Y,caches)
     
        parameters = updateParams(parameters,grads,learning_rate)            
    
    return (parameters, costs)

para = []
def ACT(x, a, b, act):
    if(act == "gaussian"):
        return np.exp(-b*np.linalg.norm(x-a))
    elif(act == "tanh"):
        num = 1 - np.exp(-(np.dot(x.T, a) + b))
        den = 1 + np.exp(-(np.dot(x.T, a) + b))
        return (num/den)

def init(l_hidden, dimensions):
    a = []
    b = []
    for i in range(l_hidden):
        a.append(np.random.rand(dimensions,1))
        b.append(np.random.rand(1))
    return (a,b)

def oneHotEnc(y):
    from sklearn.preprocessing import OneHotEncoder 
    onehotencoder = OneHotEncoder() 
    y = onehotencoder.fit_transform(y).toarray()
    return y

def compute(l_hidden,training_data_X,testing_data_X,training_data_y,testing_data_y,act):

    Y_enc = oneHotEnc(training_data_y)
    H = np.zeros((training_data_X.shape[0],l_hidden))
    for i in range(H.shape[1]):
        for j in range(H.shape[0]):
            H[j][i]=ACT(train_x[j],a[i],b[i],act)
    W=np.dot(np.linalg.pinv(H),Y_enc)
    
    H=np.zeros((testing_data_X.shape[0],l_hidden))
    for i in range(H.shape[1]):
        for j in range(H.shape[0]):
            H[j][i]=ACT(testing_data_X[j],a[i],b[i],act)

    p = np.dot(H,W)
    p = np.argmax(p,axis=1)
    p = np.reshape(p,newshape=(p.shape[0],1))
    metrics(p, testing_data_y)


dataset = pd.read_excel('data5.xslx', header = None)

row, col = dataset.shape
feats = col - 1 

# normalization
dataset.loc[:, dataset.columns != feats] = (dataset.loc[:, dataset.columns != feats]-dataset.loc[:, dataset.columns != feats].mean(axis=0))/dataset.loc[:, dataset.columns != feats].std(axis=0)

# spliting dataset into train test and val
training_data, validation_data, testing_data = np.split(dataset.sample(frac=1),[int(0.7*len(dataset)), int(0.8*len(dataset))])

training_data = np.array(training_data)
validation_data = np.array(validation_data)
testing_data = np.array(testing_data)
training_data_X = training_data[:, :feats]
training_data_y = training_data[:, feats]
validation_data_X = validation_data[:, :feats]
validation_data_y = validation_data[:, feats]
testing_data_X = testing_data[:, :feats]
testing_data_y = testing_data[:, feats]

train_row, train_col = training_data_X.shape

layers_dims = [72,32,72]
parameters, costs = layerLModel(training_data_X, training_data_y, layers_dims, num_iterations=1500, stack=False, learning_rate=0.005)
paramsAE1 = parameters

W1 = paramsAE1['W1']
b1 = paramsAE1['b1']
X_new, _ = linearActivationFunction(X.T,W1,b1,"sigmoid")

layers_dims = [32,16,32]
parameters,costs = layerLModel(training_data_X, training_data_y, layers_dims,num_iterations=1500,stack=False,learning_rate=0.05, print_cost = True)
paramsAE2 = parameters

W1 = paramsAE1['W1']
W2 = paramsAE2['W1']
b1 = paramsAE1['b1']
b2 = paramsAE2['b1']

x, _ = sigmoidFunction(np.dot(W1,X.T) + b1)
x, _ = sigmoidFunction(np.dot(W2,x) + b2)

a, b = init(256, X.shape[1])
print("Tanh Accuracy: ")
compute(256, training_data_X, testing_data_X, training_data_y, testing_data_y, "tanh")
print()
a, b = init(256,X.shape[1])
print("Gaussian Accuracy: ")
compute(256,training_data_X,testing_data_X,training_data_y,testing_data_y,"gaussian")


Tanh Accuracy: 
---------------------------------------------------------------------------
Sensitivity :  0.8181818181818182
Specificity :  0.8888888888888888
Accuracy ((TN+TP)/(TN+TP+FN+FP)) :  0.855072463768116

Gaussian Accuracy: 
---------------------------------------------------------------------------
Sensitivity :  0.9230769230769231
Specificity :  0.851063829787234
Accuracy ((TN+TP)/(TN+TP+FN+FP)) :  0.8837209302325582


#Q8
Implement support vector machine (SVM) classifier for the multi-class classification task. You can use one
vs one and one vs all multiclass coding methods to create binary SVM models. Implement the SMO
algorithm for the evaluation of the training parameters of SVM such as Lagrange multipliers. You can use
holdout approach (70%, 10%, 20%) for evaluating the performance of the classifier. The dataset
(data5.xlsx) contains 7 features and the last column is the output (class labels). Evaluate individual
accuracy and overall accuracy. You can use RBF and polynomial kernels. Evaluate the classification
performance of multiclass SVM for each kernel function. (Packages such as Scikitlearn, keras, tensorflow,
pytorch etc. are not allowed)

In [ ]:

dataset = pd.read_excel('data5.xslx', header = None)

row, col = dataset.shape
feats = col - 1 

# normalization
dataset.loc[:, dataset.columns != feats] = (dataset.loc[:, dataset.columns != feats]-dataset.loc[:, dataset.columns != feats].mean(axis=0))/dataset.loc[:, dataset.columns != feats].std(axis=0)

# spliting dataset into train test and val
training_data, validation_data, testing_data = np.split(dataset.sample(frac=1),[int(0.7*len(dataset)), int(0.8*len(dataset))])

training_data = np.array(training_data)
validation_data = np.array(validation_data)
testing_data = np.array(testing_data)
training_data_X = training_data[:, :feats]
training_data_y = training_data[:, feats]
validation_data_X = validation_data[:, :feats]
validation_data_y = validation_data[:, feats]
testing_data_X = testing_data[:, :feats]
testing_data_y = testing_data[:, feats]

train_row, train_col = training_data_X.shape


class SupportVec():
    def __init__(self, max_iter=10000, kernel_type='linear', C=1.0, epsilon=0.001):
        self.kernels = {
            'linear' : self.linearKernel
        }
        self.max_iter = max_iter
        self.kernel_type = kernel_type
        self.C = C
        self.epsilon = epsilon
    def fit(self, X, y):

        n, d = X.shape[0], X.shape[1]
        alpha = np.zeros((n))
        kernel = self.kernels[self.kernel_type]
        count = 0
        while(True):
            count += 1
            alpha_prev = np.copy(alpha)

            for j in range(0, n):
                i = self.initRandomize(0, n-1, j) # Get random int i~=j
                x_i, x_j, Yi, Yj = X[i,:], X[j,:], y[i], y[j]
                k_ij = kernel(x_i, x_i) + kernel(x_j, x_j) - 2 * kernel(x_i, x_j)
                if k_ij == 0:
                    continue
                jPrimeAlpha, iPrimeAlpha = alpha[j], alpha[i]
                (L, H) = self.computerLH(self.C, jPrimeAlpha, iPrimeAlpha, Yj, Yi)
                self.w = self.computeWeights(alpha, y, X)
                self.b = self.computeBias(X, y, self.w)   
                E_i = self.E(x_i, Yi, self.w, self.b)
                E_j = self.E(x_j, Yj, self.w, self.b)

                alpha[j] = jPrimeAlpha + float(Yj * (E_i - E_j))/k_ij
                alpha[j] = max(alpha[j], L)
                alpha[j] = min(alpha[j], H)

                alpha[i] = alpha_prime_i + y_i*Yj * (alpha_prime_j - alpha[j])

            diff = np.linalg.norm(alpha - alpha_prev)
            if diff < self.epsilon:
                break
            if(count >= self.max_iter):
                print("Iteration number exceeded the max of %d iterations" % (self.max_iter))
                return

        self.b = self.computeBias(X, y, self.w)
        if self.kernel_type == 'linear':
            self.w = self.computeWeights(alpha, y, X)

        alpha_idx = np.where(alpha > 0)[0]
        support_vectors = X[alpha_idx, :]
        return support_vectors, count

    def predict(self, X):
        return self.h(X, self.w, self.b)

    def computeBias(self, X, y, w):
        biasVar = y - np.dot(w.T, X.T)
        return np.mean(biasVar)

    def computeWeights(self, alpha, Y, X):
        return np.dot(X.T, np.multiplY(alpha,y))

    def h(self, X, weight, bias):
        return np.sign(np.dot(weight.T, X.T) + bias).astype(int)

    def E(self, Xk, Yk, weight, bias):
        return self.h(Xk, weight, bias) - Yk

    def computerLH(self, C, jPrimeAlpha, iPrimeAlpha, Yj, Yi):
        if(Yi != Yj):
            return (max (0, jPrimeAlpha - iPrimeAlpha), min(C, C - iPrimeAlpha + jPrimeAlpha))
        else:
            return (max (0, iPrimeAlpha + jPrimeAlpha - C), min(C, iPrimeAlpha + jPrimeAlpha))

    def initRandomize(self, a, b, count):

        iterations =  count

        counter  = 0

        while(iterations ==  count and counter<1000):
            iterations = random.randint(a,b)
            counter += 1
        return iterations
   
    def linearKernel(self, x1, x2):
        return np.dot(x1, x2.T)

model = SupportVec(max_iter=1000, epsilon=0.01)
model.fit(training_data_X, training_data_y)
test_pred = model.predict(test_x)
print('Testing data')
metrics(testing_data_y, test_pred)

Testing data
---------------------------------------------------------------------------
Sensitivity :  0.8653846153846154
Specificity :  0.8043478260869565
Accuracy ((TN+TP)/(TN+TP+FN+FP)) :  0.8367346938775511


#Q9
Implement a multi-channel 1D deep CNN architecture (as shown in Fig. 1) for the seven-class classification
task. The input and the class labels are given in. mat file format. There is a total of 17160 number of
instances present in both input and class-label data files. The input data for each instance is a
multichannel time series (12-channel) with size as (12 ×800). The class label for each multichannel time
series instance is given in the class_label.mat file. You can select the training and test instances using
hold-out cross-validation (70% training, 10% validation, and 20% testing). The architecture of the multichannel deep CNN is given as follows. The number of filters, length of each filter, and number of neurons
in the fully connected layers are shown in the following figure. Evaluate individual accuracy and overall
accuracy. (Packages such as Scikitlearn, keras, tensorflow, pytorch etc. are allowed)


In [ ]:
!pip install mat4py
import keras 
import tensorflow as tf
import sklearn
import numpy as np
import pandas as pd
import scipy.io as sio 
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Input, Dense, Activation, Flatten, Conv1D, Dropout,MaxPooling1D,MaxPool1D
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.utils import to_categorical
from sklearn.utils import shuffle
from mat4py import loadmat

In [ ]:
data_feats = sio.loadmat('input.mat')
data_feats = pd.DataFrame(data_feats["x"])
data_feats = (np.asarray(data_feats)).T

data_labels=sio.loadmat('class_label.mat') 
data_labels=np.asarray(data_labels["y"])

In [ ]:
X_feats = []
for i in range(len(data_feats)):
  X_feats.append(data_feats[i][0])
X_feats=np.asarray(X_feats)
X_feats=X_feats.transpose(0,2,1)
for i in range(len(X_feats)):
  X_feats[i]=preprocessing.normalize(X_feats[i])

In [ ]:
Y_labels = []
for i in range(len(data_labels)):
  Y_labels.append(data_labels[i][0]-1)
Y_labels=np.asarray(Y_labels)

In [ ]:
training_data_X, testing_data_X, training_data_Y, testing_data_Y = train_test_split(X_feats, Y_labels, test_size = 2/10, train_size = 8/10, random_state = 0)
training_data_X, validation_data_X, training_data_Y, validation_data_Y = train_test_split(training_data_X, training_data_Y, test_size=1/8, train_size=7/8, random_state = 0)

In [ ]:
model = Sequential()
model.add(Conv1D(kernel_size=7, filters=20, activation='relu', input_shape=(800,12)))
model.add(MaxPooling1D(pool_size = 3, strides = 3))
model.add(Conv1D(kernel_size = 7, filters = 60, activation = 'relu'))
model.add(MaxPooling1D(pool_size = 3, strides = 3))
model.add(Dropout(0.7))
model.add(Conv1D(filters = 120, kernel_size = 7))
model.add(Conv1D(filters = 120, kernel_size = 7))
model.add(Flatten())
model.add(Dense(2000, activation = 'relu'))
model.add(Dense(700))
model.add(Dense(50))
model.add(Dense(7, activation = 'sigmoid'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 794, 20)           1700      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 264, 20)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 258, 60)           8460      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 86, 60)           0         
 1D)                                                             
                                                                 
 dropout (Dropout)           (None, 86, 60)            0         
                                                                 
 conv1d_2 (Conv1D)           (None, 80, 120)           5

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history=model.fit(training_data_X, training_data_Y, epochs=10, batch_size=1000, validation_data = (validation_data_X, validation_data_Y))
print(history)

Epoch 1/10
13/13 [==============================] - 56s 4s/step - loss: 3.3170 - accuracy: 0.2605 - val_loss: 1.5590 - val_accuracy: 0.4266
Epoch 2/10
13/13 [==============================] - 54s 4s/step - loss: 1.1386 - accuracy: 0.6311 - val_loss: 0.5840 - val_accuracy: 0.7686
Epoch 3/10
13/13 [==============================] - 54s 4s/step - loss: 0.3544 - accuracy: 0.8902 - val_loss: 0.0763 - val_accuracy: 0.9808
Epoch 4/10
13/13 [==============================] - 54s 4s/step - loss: 0.1103 - accuracy: 0.9632 - val_loss: 0.0387 - val_accuracy: 0.9854
Epoch 5/10
13/13 [==============================] - 55s 4s/step - loss: 0.0723 - accuracy: 0.9756 - val_loss: 0.0038 - val_accuracy: 1.0000
Epoch 6/10
13/13 [==============================] - 54s 4s/step - loss: 0.0146 - accuracy: 0.9969 - val_loss: 0.0011 - val_accuracy: 1.0000
Epoch 7/10
13/13 [==============================] - 54s 4s/step - loss: 0.0055 - accuracy: 0.9992 - val_loss: 3.0739e-04 - val_accuracy: 1.0000
Epoch 8/10
13/13

#Q10 
Implement the hybrid fuzzy deep neural network (HFDNN) for the three-class classification task. The input
and output instances for the HFDNN are given in data5.xlsx file (first seven columns input and last column
is the output). For a single instance, the input size is 7. There is a total of 210 instances given in the input
and label datasets. You can select the training and test instances using hold-out cross-validation
(70%training, 10% validation, and 20% testing). The HFDNN architecture shown in Fig. 2 has neural
network hidden layers, fuzzy membership and rule layers, and a fusion layer. The descriptions of the HFDNN architecture are given in reference. Evaluate individual accuracy and overall accuracy. (Packages
such as Scikitlearn, keras, tensorflow, pytorch etc. are not allowed)

## Downloading as pdf

In [ ]:
!wget -nc https://raw.githubusercontent.com/brpy/colab-pdf/master/colab_pdf.py
from colab_pdf import colab_pdf
colab_pdf('NNFL_A2.ipynb')

--2021-12-01 14:45:55--  https://raw.githubusercontent.com/brpy/colab-pdf/master/colab_pdf.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1864 (1.8K) [text/plain]
Saving to: ‘colab_pdf.py’

colab_pdf.py        100%[===================>]   1.82K  --.-KB/s    in 0s      

2021-12-01 14:45:56 (5.88 MB/s) - ‘colab_pdf.py’ saved [1864/1864]

Mounted at /content/drive/




Extracting templates from packages: 100%
[NbConvertApp] Converting notebook /content/drive/MyDrive/Colab Notebooks/NNFL_A2.ipynb to pdf
[NbConvertApp] Writing 176274 bytes to ./notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: [u'xelatex', u'./notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: [u'bibtex', u'./notebook']
[NbConvertApp] WARNING | bibt

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

'File ready to be Downloaded and Saved to Drive'